In [95]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import psycopg2
import requests
import csv
import urllib.request
import urllib.parse
import re
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains

# 특정 지역의 관광지 추천

### 내용 추출 및 리스트로 반환

In [58]:
def storeNamePrint():
    time.sleep(0.2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    travels = soup.select('.placelist > .PlaceItem')
    count = 1
    for travel in travels:
        temp = []
        spot_name = travel.select('.head_item > .tit_name > .link_name')[0].text 
        score = travel.select('.rating > .score > .num')[0].text
        review = travel.select('.rating > .review')[0].text
        link = travel.select('.contact > .moreview')[0]['href']
        addr = travel.select('.addr')[0].text
    
        # 리뷰 개수 문자열 제거 후 숫자만 반환
        review = review[3:len(review)]

        # 리뷰 개수 쉼표 제거
        review = int(re.sub(",", "", review))

        print(spot_name, score, review, link, addr)

        temp.append(spot_name)
        temp.append(score)
        temp.append(review)
        temp.append(link)
        temp.append(addr)

        list.append(temp)

    f = open(filename + '.csv', "w", encoding="utf-8-sig", newline="")
    writercsv = csv.writer(f)
    header = ['Name', 'Score', 'Review', 'Link', 'Addr']
    writercsv.writerow(header)

    for i in list:
        writercsv.writerow(i)
   
    return list

### 데이터프레임으로 저장

In [59]:
# 데이터프레임 -> 저장
import pandas as pd

def pandas1 ():     
    df = pd.DataFrame(list, columns=['Name','Score','Review','Link','Addr']) 
    df = df.astype({'Score':'float'})                                             #별점 -> float 타입으로 변환

   
    sorted_df = df.sort_values(by=['Review'], ascending = False)                 #리뷰 개수로 내림차순 정렬
    spot_df = sorted_df[0:15]                                                     #정렬된 데이터에서 top 15개의 관광지 추출
    
    spot_list = spot_df['Name'].tolist()                                          #제목 리스트
    spot_score = spot_df['Score'].tolist()                                        #별점 리스트
    spot_review = spot_df['Review'].tolist()                                      #리뷰 개수 리스트
    spot_link = spot_df['Link'].tolist()                                          #링크 리스트
    spot_addr = spot_df['Addr'].tolist()                                          #주소 리스트
    
    # 엑셀 저장
    sorted_df.to_excel(excel_writer="C://jupyter_notebook//crawling//" + filename + ".xlsx")
    return sorted_df, spot_list, spot_score, spot_review, spot_link, spot_addr

In [60]:
# 구글 인증
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}

list = []

url = "https://map.kakao.com/"
# 현재파일에 있는 크롬 드라이버를 가져와서 열기
options = webdriver.ChromeOptions()  # 크롬 브라우저 옵션
# options.add_argument('headless') # 브라우저 안 띄우기
options.add_argument('lang=ko_KR')  # KR 언어
chromedriver_path = "C:\jupyter_notebook\crawling\chromedriver"  # 크롬 드라이버 위치
driver = webdriver.Chrome(os.path.join(os.getcwd(), 'chromedriver'), options=options)  # chromedriver 열기

# 1. 카카오 지도로 이동
driver.get(url)

searchloc = input("여행지역 입력: ")
searchloc = searchloc + ' 관광지'                                                       # 무슨무슨 관광지로 검색
filename = input("파일이름(영어): ")

# 2. 서울시 음식점 입력 후 찾기 버튼 클릭 xpath활용
search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')           # 검색 창
search_area.send_keys(searchloc)                                                        # 검색어 입력
driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색

time.sleep(2)
# 3 장소 버튼 클릭
driver.find_element_by_xpath('//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)

C:\Users\User\AppData\Local\Temp/ipykernel_3404/224664573.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.join(os.getcwd(), 'chromedriver'), options=options)  # chromedriver 열기


여행지역 입력: 제주도
파일이름(영어): jeju_0209


C:\Users\User\AppData\Local\Temp/ipykernel_3404/224664573.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')           # 검색 창
C:\Users\User\AppData\Local\Temp/ipykernel_3404/224664573.py:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
C:\Users\User\AppData\Local\Temp/ipykernel_3404/224664573.py:29: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)


> 여러 페이지

In [61]:
page = 1
page2 = 0
print("start")
# 1페이지부터 30페이지까지 출력
for i in range(0, 30):

    # 페이지 넘어가며 출력
    try:
        page2 += 1
        print("**", page, "**")

        driver.find_element_by_xpath(f'//*[@id="info.search.page.no{page2}"]').send_keys(Keys.ENTER)
        storeNamePrint()
        
        if (page2) % 5 == 0:
            driver.find_element_by_xpath('//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)
            page2 = 0

        page += 1
    except:
        break
        
        
print("end")

sorted_df, spot_list, spot_score, spot_review, spot_link, spot_addr= pandas1()

start
** 1 **


C:\Users\User\AppData\Local\Temp/ipykernel_3404/2147515281.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(f'//*[@id="info.search.page.no{page2}"]').send_keys(Keys.ENTER)


광고제주바다목장 다이브리조트 4.0 2 https://place.map.kakao.com/941661099 
제주특별자치도 제주시 한경면 한경해안로 565
(지번) 한경면 신창리 542-9

올레길 7코스(서귀포-월평 올레) 4.4 31 https://place.map.kakao.com/11635431 



올레길 6코스(쇠소깍-서귀포 올레) 5.0 26 https://place.map.kakao.com/8015963 



올레길 1코스(시흥-광치기 올레) 4.0 19 https://place.map.kakao.com/10626234 



올레길 18코스(제주원도심-조천 올레) 4.6 16 https://place.map.kakao.com/13597626 



올레길 5코스(남원-쇠소깍 올레) 4.7 11 https://place.map.kakao.com/11828668 



올레길 17코스(광령-제주원도심 올레) 5.0 16 https://place.map.kakao.com/12753473 



올레길 10코스(화순-모슬포 올레) 5.0 22 https://place.map.kakao.com/12753435 



올레길 8코스(월평-대평 올레) 4.2 17 https://place.map.kakao.com/11792904 



올레길 16코스(고내-광령 올레) 4.7 11 https://place.map.kakao.com/11943943 



올레길 20코스(김녕-하도 올레) 4.0 12 https://place.map.kakao.com/17605888 



올레길 13코스(용수-저지 올레) 4.5 13 https://place.map.kakao.com/12753454 



올레길 19코스(조천-김녕 올레) 4.1 14 https://place.map.kakao.com/14617417 



올레길 4코스(표선-남원 올레) 4.6 8 https://place.map.kakao.com/12753459 



올레길 2코스(광치기-온평 올레)

C:\Users\User\AppData\Local\Temp/ipykernel_3404/2147515281.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)


한담해변 3.7 47 https://place.map.kakao.com/19293737 
제주특별자치도 제주시 애월읍 애월리


송악산 4.8 457 https://place.map.kakao.com/7956367 
제주특별자치도 서귀포시 대정읍 상모리 산 2


월정리해수욕장 4.1 392 https://place.map.kakao.com/17600274 
제주특별자치도 제주시 구좌읍 월정리 33-3


큰엉해안경승지 4.6 278 https://place.map.kakao.com/11294440 
제주특별자치도 서귀포시 남원읍 남원리


광치기해변 4.5 539 https://place.map.kakao.com/13723499 
제주특별자치도 서귀포시 성산읍 오조리


서귀포치유의숲 4.6 102 https://place.map.kakao.com/1766125269 
제주특별자치도 서귀포시 산록남로 2271
(지번) 호근동 산 4

여미지식물원 3.1 165 https://place.map.kakao.com/8004227 
제주특별자치도 서귀포시 중문관광로 93 중문단지 내
(지번) 색달동 2484-1

새연교 4.0 109 https://place.map.kakao.com/26875121 
제주특별자치도 서귀포시 서홍동 707-4


제주동백수목원 4.0 311 https://place.map.kakao.com/236028555 
제주특별자치도 서귀포시 남원읍 위미리 927


제주성읍마을 2.9 62 https://place.map.kakao.com/12501794 
제주특별자치도 서귀포시 표선면 성읍정의현로 19
(지번) 표선면 성읍리 775-1

중문색달해수욕장 4.2 113 https://place.map.kakao.com/8069764 
제주특별자치도 서귀포시 색달동 3039


다랑쉬오름 4.8 154 https://place.map.kakao.com/25200772 
제주특별자치도 제주시 구좌읍 세화리 산 6


한담해안산책로 4.6 305 

In [55]:
sorted_df

,Name,Score,Review,Link,Addr
31,성산일출봉,4.6,964,https://place.map.kakao.com/25285071,\n제주특별자치도 서귀포시 성산읍 성산리 78\n\n
30,함덕해수욕장,4.5,956,https://place.map.kakao.com/8148451,\n제주특별자치도 제주시 조천읍 조함해안로 525\n(지번) 조천읍 함덕리 1004...
32,카멜리아힐,3.8,935,https://place.map.kakao.com/9461977,\n제주특별자치도 서귀포시 안덕면 병악로 166\n(지번) 안덕면 상창리 271\n
58,새별오름,4.3,837,https://place.map.kakao.com/25274725,\n제주특별자치도 제주시 애월읍 봉성리 산 59-8\n\n
34,협재해수욕장,4.6,711,https://place.map.kakao.com/8159415,\n제주특별자치도 제주시 한림읍 한림로 329-10\n(지번) 한림읍 협재리 2447\n
...,...,...,...,...,...
245,월평마을 아왜낭목 쉼터,4.5,0,https://place.map.kakao.com/697827717,\n제주특별자치도 서귀포시 월평동 738\n\n
341,용천동굴,3.0,0,https://place.map.kakao.com/12658130,\n제주특별자치도 제주시 구좌읍 월정리 1837-2\n\n
277,제주올레 7코스 여행자센터,5.0,0,https://place.map.kakao.com/14080397,\n제주특별자치도 서귀포시 중정로 22\n(지번) 서귀동 316-1\n
321,산물관광농원,5.0,0,https://place.map.kakao.com/211417707,\n제주특별자치도 서귀포시 남원읍 하례로 90\n(지번) 남원읍 하례리 155-1\n


# 맛집 찾기

In [102]:
a.clear()
b.clear()
c.clear()
d.clear()
e.clear()

NameError: name 'a' is not defined

In [198]:
a

[]

In [105]:
# 파일저장 함수
a=[]
b=[]
c=[]
d=[]
e=[]

def storeNamePrint2():
#    global a,b,c,d,e
    time.sleep(0.2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    cafe_lists = soup.select('.placelist > .PlaceItem')
    count = 1

    for cafe in cafe_lists:
        temp = []
    
        cafe_name = cafe.select('.head_item > .tit_name > .link_name')[0].text  # cafeName
        food_score = cafe.select('.rating > .score > .num')[0].text
        review = cafe.select('.rating > .review')[0].text
        link = cafe.select('.contact > .moreview')[0]['href']
        addr = cafe.select('.addr')[0].text
        
        
        # "리뷰" 문자열 제거 후 숫자만 반환
        review = review[3:len(review)]
        review = int(re.sub(",", "", review))
        
        a.append(cafe_name)
        b.append(food_score)
        c.append(review)
        d.append(link)
        e.append(addr)

        # 숫자만 반환된 문자를 쉼표를
        #review = int(re.sub(",", "", review))

        print(cafe_name, food_score, review, link, addr)

        temp.append(cafe_name)
        temp.append(food_score)
        temp.append(review)
        temp.append(link)
        temp.append(addr)

        list.append(temp)

#     f = open(filename + '.csv', "w", encoding="utf-8-sig", newline="")
#     writercsv = csv.writer(f)
#     header = ['Name', 'Score', 'Review', 'Link', 'Addr']
#     writercsv.writerow(header)

#     for i in list:
#         writercsv.writerow(i)

# 데이터프레임 -> 저장

def pandas2 ():
    
    data = pd.DataFrame()
    data['Name'] = a
    data['Score'] = b
    data['Review'] = c
    data['Link'] = d
    data['Addr'] = e

#     data['Review_cnt'] = data['Review'].str.split(' ', expand = True)[1]   # 리뷰에서 숫자 추출
#     data['Review_cnt'] = pd.to_numeric(data['Review_cnt'])   # 리뷰를 숫자화
#     data = data.drop(['Review'],axis = 1)  # 컬럼 삭제
#     data = data[['Name', 'Score', 'Review_cnt', 'Link', 'Addr']]   # 컬럼 위치변경

    # int + 정렬 + 5개뽑기
    data = data.astype({'Review' : int})
    data = data.sort_values(by=['Review', 'Score'], ascending = False)
    #data = data.head(5)
    
    
    # 
    Name_list = data['Name'].values.tolist()
    Score_list = data['Score'].values.tolist()
    Review_list = data['Review'].values.tolist()
    Link_list = data['Link'].values.tolist()
    Addr_list = data['Addr'].values.tolist()


    # 엑셀 저장
    data.to_excel(excel_writer="C://jupyter_notebook//크롤링//" + filename + ".xlsx")
    return data, Name_list, Score_list,  Review_list, Link_list, Addr_list


In [106]:
# 구글 인증

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}

list = []
a.clear()
b.clear()
c.clear()
d.clear()
e.clear()

url = "https://map.kakao.com/"
# 현재파일에 있는 크롬 드라이버를 가져와서 열기
options = webdriver.ChromeOptions()  # 크롬 브라우저 옵션
# options.add_argument('headless') # 브라우저 안 띄우기
options.add_argument('lang=ko_KR')  # KR 언어
chromedriver_path = "D:\pythonProject3\chromedriver"  # 크롬 드라이버 위치
driver = webdriver.Chrome(os.path.join(os.getcwd(), 'chromedriver'), options=options)  # chromedriver 열기


for spot in spot_list :
    print(spot + ' 맛집')
    driver.get(url)
    searchloc = spot + ' 맛집'
    #filename = input("저장할 파일이름 : (영어) ")

    # 2. 대구시 음식점 입력 후 찾기 버튼 클릭 xpath활용
    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    time.sleep(1)
    search_area.clear()
    time.sleep(1)
    
    search_area.send_keys(searchloc)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    time.sleep(1)
    
    # 3 장소 버튼 클릭R
    driver.find_element_by_xpath('//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)
    time.sleep(2)
    
    page = 1
    page2 = 0
    print("start")

    # 1페이지부터 34페이지까지 출력
    for i in range(0,5):
        
        # 페이지 넘어가며 출력
        try:
            page2 += 1
            print("**", page, "**")

            driver.find_element_by_xpath(f'//*[@id="info.search.page.no{page2}"]').send_keys(Keys.ENTER)

            storeNamePrint2()

            if (page2) % 5 == 0:
                driver.find_element_by_xpath('//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)
                page2 = 0

            page += 1
        except:
            break
        
        data, Name_list, Score_list,  Review_list, Link_list, Addr_list = pandas2()
    
    print("end")

C:\Users\User\AppData\Local\Temp/ipykernel_3404/2395261588.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.join(os.getcwd(), 'chromedriver'), options=options)  # chromedriver 열기


성산일출봉 맛집


C:\Users\User\AppData\Local\Temp/ipykernel_3404/2395261588.py:29: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
C:\Users\User\AppData\Local\Temp/ipykernel_3404/2395261588.py:35: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
C:\Users\User\AppData\Local\Temp/ipykernel_3404/2395261588.py:39: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)


start
** 1 **


C:\Users\User\AppData\Local\Temp/ipykernel_3404/2395261588.py:54: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(f'//*[@id="info.search.page.no{page2}"]').send_keys(Keys.ENTER)


광고제주호랭이 성산점 4.1 98 https://place.map.kakao.com/143576812 
제주특별자치도 서귀포시 성산읍 일출로 274-1 2층
(지번) 성산읍 성산리 132-2

광고삼무국수 성산점 2.9 575 https://place.map.kakao.com/1512555158 
제주특별자치도 서귀포시 성산읍 고성오조로 74-1
(지번) 성산읍 고성리 307-4

성산어촌계해녀의집 3.0 5 https://place.map.kakao.com/21499361 
제주특별자치도 서귀포시 성산읍 일출로 284-34
(지번) 성산읍 성산리

코코마마 성산점 4.0 660 https://place.map.kakao.com/787179341 
제주특별자치도 서귀포시 성산읍 일출로 258-11
(지번) 성산읍 성산리 140-8

꽃담수제버거 4.2 119 https://place.map.kakao.com/27459317 
제주특별자치도 서귀포시 성산읍 일출로 284-7
(지번) 성산읍 성산리 111

스타벅스 성산일출봉점 3.6 133 https://place.map.kakao.com/18143017 
제주특별자치도 서귀포시 성산읍 일출로 284-5 1,2층
(지번) 성산읍 성산리 109-11

성산옥탑 4.1 79 https://place.map.kakao.com/137722576 
제주특별자치도 서귀포시 성산읍 일출로 270-8 2층
(지번) 성산읍 성산리 136-13

마가리따은혜씨 3.6 184 https://place.map.kakao.com/1311422198 
제주특별자치도 서귀포시 성산읍 일출로 258-11 2층
(지번) 성산읍 성산리 140-11

성산어촌계우뭇개식당 5.0 0 https://place.map.kakao.com/23972633 
제주특별자치도 서귀포시 성산읍 일출로 284-34
(지번) 성산읍 성산리 104-1

성산해녀짬뽕 4.5 121 https://place.map.kakao.com/131159720 
제주특별자치도 서

C:\Users\User\AppData\Local\Temp/ipykernel_3404/2395261588.py:59: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)


start
** 1 **
광고훈남횟집 4.6 215 https://place.map.kakao.com/867814713 
제주특별자치도 제주시 조천읍 함덕13길 7 1층
(지번) 조천읍 함덕리 1002-21

광고몰돗 4.8 86 https://place.map.kakao.com/2003171967 
제주특별자치도 제주시 조천읍 함대로 2
(지번) 조천읍 함덕리 3037-1

광고해소랑 4.4 19 https://place.map.kakao.com/215782470 
제주특별자치도 제주시 조천읍 신북로 491 1층
(지번) 조천읍 함덕리 1015-3

델문도 3.2 872 https://place.map.kakao.com/26867476 
제주특별자치도 제주시 조천읍 조함해안로 519-10 1층
(지번) 조천읍 함덕리 1008-1

고집돌우럭 함덕점 4.9 839 https://place.map.kakao.com/28082185 
제주특별자치도 제주시 조천읍 신북로 491-9 2층
(지번) 조천읍 함덕리 1020-2

무거버거 3.5 334 https://place.map.kakao.com/1356679233 
제주특별자치도 제주시 조천읍 조함해안로 356
(지번) 조천읍 신흥리 15

오드랑베이커리 4.1 783 https://place.map.kakao.com/1578746387 
제주특별자치도 제주시 조천읍 조함해안로 552-3 1층
(지번) 조천읍 함덕리 270-1

해녀김밥 함덕 본점 3.4 307 https://place.map.kakao.com/1412534993 
제주특별자치도 제주시 조천읍 조함해안로 490 1층
(지번) 조천읍 함덕리 1252-55

깡촌흑돼지 4.4 261 https://place.map.kakao.com/27525363 
제주특별자치도 제주시 조천읍 함덕14길 2
(지번) 조천읍 함덕리 1001

은희네해장국 함덕점 3.9 156 https://place.map.kakao.com/1482074131 
제주특별자치도 제주시 

# 길찾기

### 차(택시) 이동시간 및 비용 추출 함수
#### selenium으로 동적크롤링

In [84]:
def car_distance(depart_spot, arrive_spot):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}

    #list = []

    url = "https://map.kakao.com/"
    options = webdriver.ChromeOptions()  # 크롬 브라우저 옵션
    options.add_argument('lang=ko_KR')  # KR 언어
    chromedriver_path = "C:\jupyter_notebook\crawling\chromedriver"  # 크롬 드라이버 위치
    driver = webdriver.Chrome(os.path.join(os.getcwd(), 'chromedriver'), options=options)  # chromedriver 열기

    # 1. 카카오 지도로 이동
    driver.get(url)

    driver.find_element_by_xpath('//*[@id="search.tab2"]/a').send_keys(Keys.ENTER)
 
    #출발지 입력
    depart_area = driver.find_element_by_xpath('//*[@id="info.route.waypointSuggest.input0"]')  # 검색 창
    depart_area.send_keys(depart_spot)  # 검색어 입력
    depart_area.send_keys(Keys.ENTER)

    #도착지 입력
    time.sleep(2)
    arrive_area = driver.find_element_by_xpath('//*[@id="info.route.waypointSuggest.input1"]')  # 검색 창
    arrive_area.send_keys(arrive_spot)  # 검색어 입력

    time.sleep(2)
    arrive_area.send_keys(Keys.ENTER)

    #승용차/ 택시
    time.sleep(2)
    car = driver.find_element_by_class_name('car')
    car.send_keys(Keys.ENTER)
    
    time.sleep(1)
    car_time = driver.find_element_by_xpath('//*[@id="info.flagsearch"]/div[6]/ul/li/div[1]/div/div[1]/p/span[1]')         #차 이동시간
    toll_fee = driver.find_element_by_xpath('//*[@id="info.flagsearch"]/div[6]/ul/li/div[1]/div/div[1]/div/span[1]/span')  #톨게이트비
    taxi_fee = driver.find_element_by_xpath('//*[@id="info.flagsearch"]/div[6]/ul/li/div[1]/div/div[1]/div/span[3]/span')  #택시비
    oil_fee = driver.find_element_by_xpath('//*[@id="info.flagsearch"]/div[6]/ul/li/div[1]/div/div[1]/div/span[5]/span')   #주유비
    
    #리스트 삽입
    depart_list.append(depart_spot)
    arrive_list.append(arrive_spot)
    car_time_list.append(car_time.text)
    toll_fee_list.append(int(toll_fee.text.replace(',','')))
    taxi_fee_list.append(int(taxi_fee.text.replace(',','')))
    oil_fee_list.append(int(oil_fee.text.replace(',','')))

    #눈에 보이게 출력
    print(depart_spot + ' ~ ' + arrive_spot)
    print(car_time.text, toll_fee.text, taxi_fee.text, oil_fee.text)


    return car_time_list, toll_fee_list, taxi_fee_list, oil_fee_list

### 대중교통 이동시간 추출 함수

In [92]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

def bus_distance(depart_spot, arrive_spot):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}

    #list = []

    url = "https://map.kakao.com/"
    options = webdriver.ChromeOptions()  # 크롬 브라우저 옵션
    options.add_argument('lang=ko_KR')  # KR 언어
    chromedriver_path = "C:\jupyter_notebook\crawling\chromedriver"  # 크롬 드라이버 위치
    driver = webdriver.Chrome(os.path.join(os.getcwd(), 'chromedriver'), options=options)  # chromedriver 열기

    # 1. 카카오 지도로 이동
    driver.get(url)

    driver.find_element_by_xpath('//*[@id="search.tab2"]/a').send_keys(Keys.ENTER)

    depart_area = driver.find_element_by_xpath('//*[@id="info.route.waypointSuggest.input0"]')  # 검색 창
    depart_area.send_keys(depart_spot)  # 검색어 입력
    depart_area.send_keys(Keys.ENTER)

    time.sleep(2)
    arrive_area = driver.find_element_by_xpath('//*[@id="info.route.waypointSuggest.input1"]')  # 검색 창
    arrive_area.send_keys(arrive_spot)  # 검색어 입력

    time.sleep(2)
    arrive_area.send_keys(Keys.ENTER)

    #버스 버튼 누르기
    time.sleep(1)
    bus = driver.find_element_by_class_name('transit')
    bus.send_keys(Keys.ENTER)
    
    #버스 시간 가져오기
    
    #오류1 : NoSuchElementException 발생
    try: 
        # selenium의 대기(wait) 기능으로 해결
        bus_time = WebDriverWait(driver,timeout=10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="info.flagsearch"]/div[5]/ul/li[1]/div[1]/span[1]')))
        print(bus_time.text) 
        bus_time_list.append(bus_time.text)
        
    #오류2 : 버스 데이터 없는 구간에서 TimeoutException 발생  
    #예외처리로 해결
    except TimeoutException as e:
        bus_time_list.append('NO BUS')
        return bus_time_list
        pass
    
    return bus_time_list

In [66]:
#위에서 뽑은 여행지 15개 (리뷰순)
spot_list

['성산일출봉',
 '함덕해수욕장',
 '카멜리아힐',
 '새별오름',
 '협재해수욕장',
 '금오름',
 '마노르블랑',
 '비자림',
 '우도',
 '용머리해안',
 '광치기해변',
 '천지연폭포',
 '쇠소깍',
 '송악산',
 '스누피가든']

### 여행지 간 이동거리 계산

In [94]:
depart_list = []
arrive_list = []
car_time_list =[]
toll_fee_list =[]
taxi_fee_list = []
oil_fee_list = []

bus_time_list = []

for depart_spot in spot_list:
    for arrive_spot in spot_list:
        if depart_spot != arrive_spot:                # 출발지, 도착지 겹치지 않게
            car_distance(depart_spot, arrive_spot) 
            bus_distance(depart_spot, arrive_spot)

C:\Users\User\AppData\Local\Temp/ipykernel_3404/97708282.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.join(os.getcwd(), 'chromedriver'), options=options)  # chromedriver 열기
C:\Users\User\AppData\Local\Temp/ipykernel_3404/97708282.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="search.tab2"]/a').send_keys(Keys.ENTER)
C:\Users\User\AppData\Local\Temp/ipykernel_3404/97708282.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  depart_area = driver.find_element_by_xpath('//*[@id="info.route.waypointSuggest.input0"]')  # 검색 창
C:\Users\User\AppData\Local\Temp/ipykernel_3404/97708282.py:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  arrive_area = driver.find_element_by_xpath('//*[@id="info.route.waypointS

성산일출봉 ~ 함덕해수욕장
48분 0 27,500 5,038


C:\Users\User\AppData\Local\Temp/ipykernel_3404/2891275381.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.join(os.getcwd(), 'chromedriver'), options=options)  # chromedriver 열기


1시간 31분
성산일출봉 ~ 카멜리아힐
1시간 38분 0 56,300 10,751
2시간 40분
성산일출봉 ~ 새별오름
1시간 33분 0 57,200 10,829
2시간 39분
성산일출봉 ~ 협재해수욕장
1시간 54분 0 65,100 12,461
2시간 38분
성산일출봉 ~ 금오름
1시간 41분 0 63,600 12,100
3시간 7분
성산일출봉 ~ 마노르블랑
1시간 50분 0 69,700 13,309
3시간 3분
성산일출봉 ~ 비자림
29분 0 17,200 3,013
1시간 5분
성산일출봉 ~ 우도
14분 0 7,000 628
성산일출봉 ~ 용머리해안
1시간 53분 0 70,500 13,466
2시간 43분
성산일출봉 ~ 광치기해변
7분 0 4,200 487
31분
성산일출봉 ~ 천지연폭포
1시간 21분 0 40,600 7,596
2시간 29분
성산일출봉 ~ 쇠소깍
1시간 8분 0 35,900 6,686
1시간 58분
성산일출봉 ~ 송악산
1시간 57분 0 74,400 14,251
3시간 14분
성산일출봉 ~ 스누피가든
27분 0 15,000 2,574
57분
함덕해수욕장 ~ 성산일출봉
52분 0 27,400 5,038
1시간 36분
함덕해수욕장 ~ 카멜리아힐
1시간 12분 0 45,500 8,538
1시간 42분
함덕해수욕장 ~ 새별오름
1시간 0 36,400 6,733
1시간 46분
함덕해수욕장 ~ 협재해수욕장
1시간 21분 0 44,300 8,365
2시간 12분
함덕해수욕장 ~ 금오름
1시간 9분 0 42,900 8,020
2시간 15분
함덕해수욕장 ~ 마노르블랑
1시간 17분 0 49,000 9,228
2시간 10분
함덕해수욕장 ~ 비자림
35분 0 20,600 3,704
46분
함덕해수욕장 ~ 우도
59분 0 30,600 5,258
함덕해수욕장 ~ 용머리해안
1시간 21분 0 49,800 9,370
1시간 55분
함덕해수욕장 ~ 광치기해변
52분 0 27,100 5,007
1시간 16분
함덕해수욕장 ~ 천지연폭포
1시간 14분 0 39,700 7,

In [109]:
print(car_time_list)
print(taxi_fee_list)
print(oil_fee_list)
print(toll_fee_list)
print(depart_list)
print(bus_time_list)

['48분', '1시간 38분', '1시간 33분', '1시간 54분', '1시간 41분', '1시간 50분', '29분', '14분', '1시간 53분', '7분', '1시간 21분', '1시간 8분', '1시간 57분', '27분', '52분', '1시간 12분', '1시간', '1시간 21분', '1시간 9분', '1시간 17분', '35분', '59분', '1시간 21분', '52분', '1시간 14분', '1시간 6분', '1시간 24분', '30분', '1시간 37분', '1시간 11분', '16분', '34분', '17분', '22분', '1시간 21분', '1시간 49분', '26분', '1시간 35분', '38분', '49분', '30분', '1시간 11분', '1시간 34분', '1시간 4분', '15분', '24분', '12분', '20분', '1시간 14분', '1시간 45분', '24분', '1시간 31분', '44분', '55분', '27분', '1시간 4분', '1시간 52분', '1시간 22분', '36분', '27분', '18분', '38분', '1시간 31분', '2시간 3분', '42분', '1시간 48분', '1시간 3분', '1시간 14분', '43분', '1시간 22분', '1시간 40분', '1시간 10분', '17분', '14분', '17분', '22분', '1시간 19분', '1시간 52분', '26분', '1시간 37분', '46분', '56분', '30분', '1시간 9분', '1시간 48분', '1시간 18분', '22분', '22분', '38분', '23분', '1시간 27분', '1시간 59분', '10분', '1시간 44분', '51분', '1시간 2분', '14분', '1시간 18분', '31분', '34분', '1시간 23분', '1시간 11분', '1시간 33분', '1시간 19분', '1시간 27분', '37분', '1시간 31분', '29분', '1시간 16분', '1시간 7분', '1시간 34분

In [110]:
# 시간이 '1시간 20분' 형식으로 나와서 
# 분단위로 변경하여 새로운 리스트에 append

total_car_time_list = []                                          #차 이동시간 리스트
total_bus_time_list = []                                          #버스 이동시간 리스트

for car in car_time_list:
    print(car)
    if ('시간' in car) & ('분' in car):                                             #1시간 이상 걸리는 경우
        hour, minute = car.split()                                #시간 / 분 나눠서 다른 변수에 저장
        hour = int(hour.replace('시간',''))                       # '시간' 제거
        minute = int(minute.replace('분', ''))                    # '분'   제거
        total_car_time_list.append(hour*60+minute)                # (시간) x 60(분) 해서 분단위로 변경
        
    elif ('시간' in car) :
        total_car_time_list.append(int(car.replace('시간', ''))*60)
        
    else:
        total_car_time_list.append(int(car.replace('분', '')))
        
for bus in bus_time_list:
    if ('시간' in bus) & ('분' in bus):
        hour, minute = bus.split()
        hour = int(hour.replace('시간',''))
        minute = int(minute.replace('분', ''))
        total_bus_time_list.append(hour*60+minute)
    elif ('시간' in bus) :
        total_bus_time_list.append(int(bus.replace('시간', ''))*60)
    elif bus == 'NO BUS' :                                        #버스 없는 구간은 'NO BUS'로 리스트에 삽입
        total_bus_time_list.append(100000)
    else:
        total_bus_time_list.append(int(bus.replace('분', '')))
        
print(total_car_time_list)
print(total_bus_time_list)

48분
1시간 38분
1시간 33분
1시간 54분
1시간 41분
1시간 50분
29분
14분
1시간 53분
7분
1시간 21분
1시간 8분
1시간 57분
27분
52분
1시간 12분
1시간
1시간 21분
1시간 9분
1시간 17분
35분
59분
1시간 21분
52분
1시간 14분
1시간 6분
1시간 24분
30분
1시간 37분
1시간 11분
16분
34분
17분
22분
1시간 21분
1시간 49분
26분
1시간 35분
38분
49분
30분
1시간 11분
1시간 34분
1시간 4분
15분
24분
12분
20분
1시간 14분
1시간 45분
24분
1시간 31분
44분
55분
27분
1시간 4분
1시간 52분
1시간 22분
36분
27분
18분
38분
1시간 31분
2시간 3분
42분
1시간 48분
1시간 3분
1시간 14분
43분
1시간 22분
1시간 40분
1시간 10분
17분
14분
17분
22분
1시간 19분
1시간 52분
26분
1시간 37분
46분
56분
30분
1시간 9분
1시간 48분
1시간 18분
22분
22분
38분
23분
1시간 27분
1시간 59분
10분
1시간 44분
51분
1시간 2분
14분
1시간 18분
31분
34분
1시간 23분
1시간 11분
1시간 33분
1시간 19분
1시간 27분
37분
1시간 31분
29분
1시간 16분
1시간 7분
1시간 34분
13분
15분
57분
1시간 49분
1시간 44분
2시간 6분
1시간 53분
2시간
38분
2시간 9분
17분
1시간 6분
1시간 18분
2시간 8분
38분
1시간 49분
1시간 21분
25분
24분
40분
26분
8분
1시간 29분
2시간 1분
1시간 46분
53분
1시간 3분
10분
1시간 20분
4분
49분
1시간 40분
1시간 36분
1시간 56분
1시간 44분
1시간 52분
30분
14분
1시간 56분
1시간 24분
1시간 11분
2시간
29분
1시간 15분
1시간 14분
35분
48분
1시간 6분
48분
49분
1시간 12분
1시간 6분
54분
1시간 11분
15분
1시간 1

# DB

In [62]:
import psycopg2

In [100]:
# postgre 접속

conn_str = "host=localhost dbname=postgres user=postgres password=admin port = 5432"
cur = None
try:
    conn = psycopg2.connect(conn_str)
    cur = conn.cursor()
    print("=== 접속 성공 =====")
# except:
# print("접속오류 !!")
except psycopg2.DatabaseError as db_err:
    print("접속오류 !!")
    print(db_err)

=== 접속 성공 =====


In [65]:
# spot 저장

for i in range(len(spot_list)) :
    
    try:
        cur.execute("INSERT INTO travel_spot (Name, Score, Review, Link, Addr) VALUES (%s, %s, %s, %s, %s);",
        (spot_list[i], spot_score[i], spot_review[i], spot_link[i], spot_addr[i])) # sql 문장
        conn.commit()
        print("----- insert ok")
    except Exception as f:
        print(f)
        conn.rollback()
        print("--- DB error")

----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok


In [108]:
# 맛집 저장

for i in range(len(Name_list)) :
    
    try:
        cur.execute("INSERT INTO kakaomap (Name, Score, Review_cnt, Link, Addr) VALUES (%s, %s, %s, %s, %s);",
        (Name_list[i], Score_list[i], Review_list[i], Link_list[i], Addr_list[i])) # sql 문장
        conn.commit()
        print("----- insert ok")
    except Exception as f:
        print(f)
        conn.rollback()
        print("--- DB error")

----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- in

In [111]:
#거리 저장

for i in range(len(depart_list)) :
    
    try:
        cur.execute("INSERT INTO distance (Depart, Arrive, Car_Time, Taxi_Fee, Oil_Fee, Toll_Fee, Bus_Time) VALUES (%s, %s, %s, %s, %s, %s, %s);",
        (depart_list[i], arrive_list[i], total_car_time_list[i], taxi_fee_list[i], oil_fee_list[i], toll_fee_list[i], total_bus_time_list[i])) # sql 문장
        conn.commit()
        print("----- insert ok")
    except Exception as f:
        print(f)
        conn.rollback()
        print("--- DB error")

----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- insert ok
----- in